# Sunbird RC API - Water Facility Demo (Public URL)

This notebook demonstrates how to interact with Sunbird RC Registry API for Water Facilities
deployed at https://sunbird-rc.akvotest.org.

## Prerequisites
- Sunbird RC services deployed at https://sunbird-rc.akvotest.org
- WaterFacility schema configured
- Keycloak `admin-api` client secret (from k8s secret `sunbird-rc`)

## Setup

In [1]:
import requests
import json
import os
import pandas as pd
from getpass import getpass
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
DOMAIN = "https://sunbird-rc.akvotest.org"
BASE_URL = f"{DOMAIN}/api/v1"
KEYCLOAK_URL = f"{DOMAIN}/auth/realms/sunbird-rc/protocol/openid-connect/token"

# Auth credentials (demo-api client)
CLIENT_ID = "demo-api"
CLIENT_SECRET = os.environ.get("SUNBIRD_DEMO_API_CLIENT_SECRET") or getpass("Enter demo-api client secret: ")

# Track created facilities during this demo session for cleanup
CREATED_OSIDS = []

print(f"Registry API: {BASE_URL}")
print(f"Keycloak Token URL: {KEYCLOAK_URL}")

Registry API: https://sunbird-rc.akvotest.org/api/v1
Keycloak Token URL: https://sunbird-rc.akvotest.org/auth/realms/sunbird-rc/protocol/openid-connect/token


## 0. Authenticate with Keycloak

Obtain an access token using the `admin-api` client credentials.

In [14]:
token_response = requests.post(KEYCLOAK_URL, data={
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials"
})

if token_response.status_code == 200:
    ACCESS_TOKEN = token_response.json()["access_token"]
    AUTH_HEADERS = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }
    print(f"Token obtained (expires in {token_response.json()['expires_in']}s)")
else:
    print(f"Auth failed: {token_response.status_code}")
    print(token_response.text)

Token obtained (expires in 300s)


## 1. Check Registry Health

In [3]:
response = requests.get(f"{DOMAIN}/health")
health = response.json()

print(f"Status: {health['result']['healthy']}")
print(f"Service: {health['result']['name']}")
display(JSON(health, expanded=True))

Status: True
Service: sunbirdrc-registry-api


<IPython.core.display.JSON object>

## 2. Total Water Facilities

In [16]:
response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json={"filters": {}, "limit": 1}
)
data = response.json()

print(f"Total Water Facilities in Registry: {data['totalCount']}")

Total Water Facilities in Registry: 18948


## 3. Create a New Water Facility

In [5]:
new_facility = {
    "geoCode": "qqgtest01",
    "waterPointType": "Protected dug well",
    "location": {
        "county": "Jawa Timur",
        "district": "Surabaya",
        "community": "Gubeng",
        "coordinates": {
            "lat": -7.2654,
            "lon": 112.7521,
            "elevation": 5.0
        }
    },
    "extractionType": "Manual",
    "pumpType": "Afridev",
    "hasDepthInfo": True,
    "depthMetres": 15.0,
    "installer": "NGO",
    "owner": "Community",
    "funder": "USAID",
    "photoUrl": "https://images.unsplash.com/photo-1581244277943-fe4a9c777189?w=800"
}

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=AUTH_HEADERS,
    json=new_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    CREATED_OSIDS.append(osid)  # Track for cleanup
    print("Water Facility created successfully!")
    print(f"osid: {osid}")

    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
    if fetch_response.status_code == 200:
        created_facility = fetch_response.json()
        print(f"wfId: {created_facility.get('wfId', 'N/A')}")
        print(f"\nGenerated wfId format: WF-<COUNTY>-<DISTRICT>-<TYPE>-<HASH>")
        display(JSON(created_facility, expanded=True))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Error: 500
{"id":"sunbird-rc.registry.post","ver":"1.0","ets":1771360084561,"params":{"resmsgid":"","msgid":"f8ee16ff-7580-4cb3-8ef2-d9ae6b433916","err":"","status":"UNSUCCESSFUL","errmsg":"dev.sunbirdrc.registry.exception.UniqueIdentifierException: dev.sunbirdrc.registry.exception.UniqueIdentifierException$GenerateException: Unable to generate id: Duplicate WaterFacility: A water point with wfId 'WF-JAW-SUR-PDW-2CD027' already exists. Water points with the same geoCode, type, and location are not allowed."},"responseCode":"OK","result":{}}


## 4. Get a Specific Water Facility by a Filter

In [6]:
response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json={"filters": [{"geoCode": new_facility.get("geoCode")}], "limit": 1}
)
data = response.json()

if data['totalCount'] > 0:
    facility = data['data'][0]
    facility_id = facility['osid']
    print(f"Fetching water facility: {facility_id}\n")
    response = requests.get(f"{BASE_URL}/WaterFacility/{facility_id}", headers=AUTH_HEADERS)
    facility = response.json()
    display(JSON(facility, expanded=True))
else:
    print("No facilities found")

Fetching water facility: 1-1c8618cb-70c2-4149-8400-211c486fec6a



<IPython.core.display.JSON object>

## 5. Update a Water Facility

In [7]:
response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json={"filters": {"geoCode": {"eq": new_facility.get("geoCode")}}, "limit": 1}
)
data = response.json()

if data['totalCount'] > 0:
    facility = data['data'][0]
    facility_id = facility['osid']

    update_data = facility.copy()
    update_data['depthMetres'] = 25.0
    update_data['funder'] = 'Updated Funder Organization'

    response = requests.put(
        f"{BASE_URL}/WaterFacility/{facility_id}",
        headers=AUTH_HEADERS,
        json=update_data
    )

    if response.status_code == 200:
        print("Water Facility updated successfully!")
        display(JSON(response.json(), expanded=True))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
else:
    print("No facilities available to update")

Water Facility updated successfully!


<IPython.core.display.JSON object>

## 6. Search Water Facilities

In [8]:
search_query = {
    "filters":{"geoCode": {"eq":new_facility.get("geoCode")}},
    "limit": 1
}

response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json=search_query
)

if response.status_code == 200:
    results = response.json()
    print(f"Found {results['totalCount']} Protected dug wells\n")

    if results['totalCount'] > 0:
        df = pd.DataFrame(results['data'])
        desired_columns = ['wfId', 'geoCode', 'waterPointType', 'extractionType', 'pumpType', 'owner']
        columns = [col for col in desired_columns if col in df.columns]
        display(df[columns])

        if 'location' in df.columns:
            print("\nLocation Details:")
            location_df = pd.json_normalize(df['location'])
            location_df['geoCode'] = df['geoCode'].values
            if 'wfId' in df.columns:
                location_df['wfId'] = df['wfId'].values
                display(location_df[['wfId', 'geoCode', 'county', 'district', 'community']])
            else:
                display(location_df[['geoCode', 'county', 'district', 'community']])
    else:
        print("No results found")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Found 1 Protected dug wells



,wfId,geoCode,waterPointType,extractionType,pumpType,owner
0,WF-JAW-SUR-PDW-2CD027,qqgtest01,Protected dug well,Manual,Afridev,Community



Location Details:


,wfId,geoCode,county,district,community
0,WF-JAW-SUR-PDW-2CD027,qqgtest01,Jawa Timur,Surabaya,Gubeng


## 7. Bulk Create Water Facilities

In [9]:
# Sample bulk data with Indonesian water points
bulk_facilities = [
    {
        "geoCode": "qqg8v2x",
        "waterPointType": "Tube well or borehole",
        "location": {
            "county": "Jawa Barat",
            "district": "Bandung",
            "community": "Dago",
            "coordinates": {"lat": -6.8649, "lon": 107.6177, "elevation": 768.0}
        },
        "extractionType": "Manual",
        "pumpType": "India Mark",
        "hasDepthInfo": False,
        "installer": "Government",
        "owner": "Community",
        "funder": "PDAM",
        "photoUrl": "https://images.unsplash.com/photo-1594398901394-4e34939a4fd0?w=800"
    },
    {
        "geoCode": "qqg9k4m",
        "waterPointType": "Tube well or borehole",
        "location": {
            "county": "Jawa Tengah",
            "district": "Semarang",
            "community": "Tembalang",
            "coordinates": {"lat": -7.0478, "lon": 110.4381, "elevation": 156.0}
        },
        "extractionType": "Electrical",
        "pumpType": "Kardia",
        "hasDepthInfo": True,
        "depthMetres": 85.0,
        "installer": "NGO",
        "owner": "Health Facility",
        "funder": "World Bank",
        "photoUrl": "https://images.unsplash.com/photo-1541252260730-0412e8e2108e?w=800"
    },
    {
        "geoCode": "qqh2n7p",
        "waterPointType": "Protected dug well",
        "location": {
            "county": "Yogyakarta",
            "district": "Sleman",
            "community": "Condongcatur",
            "coordinates": {"lat": -7.7590, "lon": 110.3926, "elevation": 142.0}
        },
        "extractionType": "Manual",
        "pumpType": "Afridev",
        "hasDepthInfo": False,
        "installer": "NGO",
        "owner": "Health Facility",
        "funder": "UNICEF",
        "photoUrl": "https://images.unsplash.com/photo-1584813539806-5f33c555f797?w=800"
    },
    {
        "geoCode": "qr4w8ty",
        "waterPointType": "Rainwater (harvesting)",
        "location": {
            "county": "Nusa Tenggara Timur",
            "district": "Kupang",
            "community": "Oesapa",
            "coordinates": {"lat": -10.1632, "lon": 123.5850, "elevation": 45.0}
        },
        "numTaps": 2.0,
        "hasDepthInfo": False,
        "installer": "Government",
        "owner": "Community",
        "funder": "AusAID",
        "photoUrl": "https://images.unsplash.com/photo-1559825481-12a05cc00344?w=800"
    },
    {
        "geoCode": "qqg5r3z",
        "waterPointType": "Protected spring",
        "location": {
            "county": "Bali",
            "district": "Gianyar",
            "community": "Ubud",
            "coordinates": {"lat": -8.5069, "lon": 115.2625, "elevation": 325.0}
        },
        "installer": "Private",
        "owner": "Private Individual",
        "funder": "Private",
        "photoUrl": "https://images.unsplash.com/photo-1504973960431-1c467e159aa4?w=800"
    }
]

print(f"Creating {len(bulk_facilities)} water facilities...\n")

results = []
for i, facility in enumerate(bulk_facilities, 1):
    response = requests.post(
        f"{BASE_URL}/WaterFacility",
        headers=AUTH_HEADERS,
        json=facility
    )

    if response.status_code == 200:
        result = response.json()
        osid = result['result']['WaterFacility']['osid']
        CREATED_OSIDS.append(osid)  # Track for cleanup

        fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
        wfId = "N/A"
        if fetch_response.status_code == 200:
            created = fetch_response.json()
            wfId = created.get('wfId', 'N/A')

        print(f"{i}. {facility['geoCode']} ({facility['waterPointType']})")
        print(f"   wfId: {wfId}")
        print(f"   osid: {osid}\n")
        results.append({'status': 'success', 'osid': osid, 'wfId': wfId, 'geoCode': facility['geoCode']})
    else:
        print(f"{i}. {facility['geoCode']} - FAILED ({response.status_code})")
        print(f"   Error: {response.text}\n")
        results.append({'status': 'failed', 'geoCode': facility['geoCode'], 'error': response.text})

print(f"{'='*50}")
print(f"Created {len([r for r in results if r['status'] == 'success'])} facilities")
print(f"Failed {len([r for r in results if r['status'] == 'failed'])} facilities")
print(f"\nTotal tracked for cleanup: {len(CREATED_OSIDS)} facilities")

Creating 5 water facilities...

1. qqg8v2x (Tube well or borehole)
   wfId: WF-JAW-BAN-TWB-E7C9BA
   osid: 1-dbe04f75-c89e-4e59-9508-295c66bc22b6

2. qqg9k4m (Tube well or borehole)
   wfId: WF-JAW-SEM-TWB-223964
   osid: 1-2d345e3c-25e2-4d5f-9343-1c4b4e6bb65b

3. qqh2n7p (Protected dug well)
   wfId: WF-YOG-SLE-PDW-49B548
   osid: 1-0f7a411c-4471-4e5c-98df-5c0ffd31043e

4. qr4w8ty (Rainwater (harvesting))
   wfId: WF-NUS-KUP-RWH-2A2B83
   osid: 1-568b13aa-4868-4a78-844c-ac00994f798b

5. qqg5r3z (Protected spring)
   wfId: WF-BAL-GIA-PS-95DB60
   osid: 1-63ebee03-0964-4e00-9aa9-5c6434a7b557

Created 5 facilities
Failed 0 facilities

Total tracked for cleanup: 5 facilities


## 7.1 Test Duplicate Rejection (wfId uniqueness)

The wfId is generated based on a hash of facility attributes (geoCode, waterPointType, location).
Attempting to create a facility with the same attributes should be rejected due to the unique constraint on wfId.

In [10]:
duplicate_facility = {
    "geoCode": "qqg8v2x",  # Same geoCode as bulk create
    "waterPointType": "Tube well or borehole",  # Same type
    "location": {
        "county": "Jawa Barat",  # Same location
        "district": "Bandung",
        "community": "Dago",
        "coordinates": {"lat": -6.8649, "lon": 107.6177, "elevation": 768.0}
    },
    "extractionType": "Electrical",  # Different extraction - should not matter
    "pumpType": "Afridev",  # Different pump - should not matter
    "installer": "Private",  # Different installer - should not matter
    "owner": "Private Individual",  # Different owner - should not matter
    "funder": "Different Funder"  # Different funder - should not matter
}

print("Testing duplicate rejection...")
print(f"Attempting to create: {duplicate_facility['geoCode']}")
print(f"Location: {duplicate_facility['location']['county']}, {duplicate_facility['location']['district']}")
print()

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=AUTH_HEADERS,
    json=duplicate_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    CREATED_OSIDS.append(osid)  # Track for cleanup (unexpected case)
    print(f"Unexpected: Facility was created with osid: {osid}")

    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
    if fetch_response.status_code == 200:
        created = fetch_response.json()
        print(f"wfId: {created.get('wfId', 'N/A')}")
else:
    print(f"Expected behavior: Duplicate rejected!")
    print(f"Status code: {response.status_code}")
    try:
        error_detail = response.json()
        print(f"Error: {json.dumps(error_detail, indent=2)}")
    except:
        print(f"Error: {response.text}")

Testing duplicate rejection...
Attempting to create: qqg8v2x
Location: Jawa Barat, Bandung

Expected behavior: Duplicate rejected!
Status code: 500
Error: {
  "id": "sunbird-rc.registry.post",
  "ver": "1.0",
  "ets": 1771360137618,
  "params": {
    "resmsgid": "",
    "msgid": "9f65806c-f76b-4289-9a26-6c85ed836c69",
    "err": "",
    "status": "UNSUCCESSFUL",
    "errmsg": "dev.sunbirdrc.registry.exception.UniqueIdentifierException: dev.sunbirdrc.registry.exception.UniqueIdentifierException$GenerateException: Unable to generate id: Duplicate WaterFacility: A water point with wfId 'WF-JAW-BAN-TWB-E7C9BA' already exists. Water points with the same geoCode, type, and location are not allowed."
  },
  "responseCode": "OK",
  "result": {}
}


## 8. Export to CSV

In [15]:
response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json={
        "filters": {
            "location.county": {"or":[
                c["location"]["county"]
                for c in bulk_facilities
            ]}
        }
    }
)
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])

    if 'location' in df.columns:
        location_df = pd.json_normalize(df['location'])
        location_df.columns = [f'location_{col}' for col in location_df.columns]
        df = pd.concat([df.drop('location', axis=1), location_df], axis=1)

    export_columns = [
        'geoCode', 'wfId', 'waterPointType', 'extractionType', 'pumpType',
        'location_county', 'location_district', 'location_community',
        'location_coordinates.lat', 'location_coordinates.lon', 'location_coordinates.elevation',
        'numTaps', 'hasDepthInfo', 'depthMetres',
        'installer', 'owner', 'funder', 'photoUrl',
        'osid', 'osCreatedAt'
    ]
    available_columns = [col for col in export_columns if col in df.columns]

    filename = f'water_facilities_export.csv'
    df[available_columns].to_csv(filename, index=False)

    print(f"Exported {len(df)} water facilities to: {filename}")
    display(df[available_columns].head())
else:
    print("No water facilities to export")

No water facilities to export


## 9. Clean Up Demo Data

This section deletes ONLY the water facilities created during this demo session (tracked in `CREATED_OSIDS`).
Production data is preserved.

In [12]:
if CREATED_OSIDS:
    print(f"Cleaning up {len(CREATED_OSIDS)} demo facilities...\n")

    deleted = 0
    failed = 0
    for osid in CREATED_OSIDS:
        response = requests.delete(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)

        if response.status_code == 200:
            print(f"Deleted: {osid}")
            deleted += 1
        else:
            print(f"Failed to delete: {osid} (Status: {response.status_code})")
            failed += 1

    print(f"\n{'='*50}")
    print(f"Deleted {deleted} demo facilities")
    if failed > 0:
        print(f"Failed {failed} facilities")
    # Clear the tracking list
    CREATED_OSIDS.clear()
else:
    print("No demo facilities to clean up")

Cleaning up 5 demo facilities...

Failed to delete: 1-dbe04f75-c89e-4e59-9508-295c66bc22b6 (Status: 500)
Failed to delete: 1-2d345e3c-25e2-4d5f-9343-1c4b4e6bb65b (Status: 500)
Failed to delete: 1-0f7a411c-4471-4e5c-98df-5c0ffd31043e (Status: 500)
Failed to delete: 1-568b13aa-4868-4a78-844c-ac00994f798b (Status: 500)
Failed to delete: 1-63ebee03-0964-4e00-9aa9-5c6434a7b557 (Status: 500)

Deleted 0 demo facilities
Failed 5 facilities


## Summary

This notebook demonstrated:
- Checking registry health
- Listing all water facilities
- Creating new water facilities with auto-generated wfId
- Reading specific facilities
- Updating facilities
- Searching facilities by type
- Bulk operations with wfId display
- Testing duplicate rejection (wfId uniqueness)
- Exporting to CSV
- Cleaning up demo data only (production data preserved)

### wfId Generation

The `wfId` is automatically generated when creating a WaterFacility using the format:
```
WF-<COUNTY_ABBR>-<DISTRICT_ABBR>-<TYPE_CODE>-<HASH>
```

**Example:** `WF-NIM-BUU-TWB-CB37DF`

**Components:**
- `WF-` - Fixed prefix for Water Facility
- `COUNTY_ABBR` - First 3 alphanumeric characters of county (uppercase)
- `DISTRICT_ABBR` - First 3 alphanumeric characters of district (uppercase)
- `TYPE_CODE` - Water point type code:
  - `PDW` - Protected dug well
  - `UDW` - Unprotected dug well
  - `TWB` - Tube well or borehole
  - `PS` - Protected spring
  - `US` - Unprotected spring
  - `PWD` - Piped water into dwelling/plot/yard
  - `PTS` - Public tap/standpipe
  - `UEB` - Unequipped borehole
  - `RWH` - Rainwater (harvesting)
  - `SSD` - Sand/Sub-surface dam (with well or standpipe)
  - `OTH` - Other
- `HASH` - 6-character SHA-256 hash of normalized attributes

**Hash inputs (normalized to lowercase, trimmed):**
- geoCode
- waterPointType
- location.county
- location.district
- location.community

**Uniqueness:** The wfId is marked as a unique index field. Duplicate facilities (same core attributes) will be rejected.

### Water Facility Schema Fields

**Required Fields:**
- `geoCode` - Unique geographic code identifier
- `location` - Administrative location object
  - `county` - County name
  - `district` - District name
  - `community` - Community name
  - `coordinates` - Geographic coordinates (optional)
    - `lat` - Latitude
    - `lon` - Longitude
    - `elevation` - Elevation in meters
- `waterPointType` - Type of water point (enum)

**Auto-generated Fields:**
- `wfId` - System-generated unique ID (format: WF-XXX-XXX-TYPE-HASH)

**Optional Fields:**
- `extractionType` - Water extraction method (Manual, Electrical, Solar, Other)
- `extractionTypeOther` - Other extraction type specification
- `pumpType` - Type of pump installed (Afridev, Consallen, India Mark, Kardia, Rope pump, Vergnet, Other)
- `pumpTypeOther` - Other pump type specification
- `numTaps` - Number of taps
- `hasDepthInfo` - Whether depth information is available
- `depthMetres` - Depth in metres
- `installer` - Entity that installed (Government, NGO, Private, Other)
- `installerOther` - Other installer specification
- `owner` - Owner of the water point (Community, Private Individual, School, NGO, Health Facility, Other institution, CBO, Private, Unknown, Other)
- `funder` - Funding organization or entity
- `photoUrl` - URL to photo of the water point

### Water Point Types Supported
- Protected dug well (PDW)
- Unprotected dug well (UDW)
- Tube well or borehole (TWB)
- Protected spring (PS)
- Unprotected spring (US)
- Piped water into dwelling/plot/yard (PWD)
- Public tap/standpipe (PTS)
- Unequipped borehole (UEB)
- Rainwater (harvesting) (RWH)
- Sand/Sub-surface dam (with well or standpipe) (SSD)
- Other (OTH)

For more information, see the WaterFacility schema at `java/registry/src/main/resources/public/_schemas/WaterFacility.json`